# Importing Data and dependencies

In [ ]:
seed = 42

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['MPLCONFIGDIR'] = os.getcwd()+'/configs/'


import numpy as np

np.random.seed(seed)



import random
random.seed(seed)
import logging
# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras import layers
tf.autograph.set_verbosity(0)
tf.get_logger().setLevel(logging.ERROR)
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)
print(tf.__version__)

# Import other libraries
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#for gan...

from tqdm import tqdm

import copy
def add_noise(cop):
    imagee = copy.copy(cop)
    var = random.randint(0, 1)
    imagee = orthogonal_rot(imagee)
    if var == 1:
        '''Add random noise to an image'''
        imagee = imagee/255
        VARIABILITY = 0.01
        deviation = VARIABILITY
        noise = np.random.normal(0, deviation, imagee.shape)
        imagee += noise
        # np.clip(img, 0., 255.)
        # return imagee*255
    # else:
        # imagee = imagee*255
        # lista = [10, 20, 25, 30, 35]

        # l = random.choice(lista)
        # box = np.zeros(l**2).reshape(l, l)
        # xx, yy = np.random.randint(0,95-l, size=2)
        # for i in range(3):
        #     imagee[xx:xx+l, yy:yy+l, i] = box

        return imagee*255
    else:
        return imagee

def orthogonal_rot(cop):
    imagee = copy.copy(cop)
    return np.rot90(imagee, np.random.choice([-1, 0, 1]))



class for customed learning rate scheduler 

In [ ]:
import time
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
class LR_ASK(tf.keras.callbacks.Callback):
    def __init__ (self, model, epochs,  ask_epoch, dwell=True, factor=.75): # initialization of the callback
        super(LR_ASK, self).__init__()
        self.model=model
        self.ask_epoch=ask_epoch
        self.epochs=epochs
        self.ask=True # if True query the user on a specified epoch
        self.lowest_vloss=np.inf
        self.lowest_aloss=np.inf
        self.best_weights=self.model.get_weights() # set best weights to model's initial weights
        self.best_epoch=1
        self.plist=[]
        self.alist=[]
        self.dwell= dwell
        self.factor=factor

    def get_list(self): # define a function to return the list of % validation change
        return self.plist, self.alist
    def on_train_begin(self, logs=None): # this runs on the beginning of training
        if self.ask_epoch == 0:
            print('you set ask_epoch = 0, ask_epoch will be set to 1', flush=True)
            self.ask_epoch=1
        if self.ask_epoch >= self.epochs: # you are running for epochs but ask_epoch>epochs
            print('ask_epoch >= epochs, will train for ', epochs, ' epochs', flush=True)
            self.ask=False # do not query the user
        if self.epochs == 1:
            self.ask=False # running only for 1 epoch so do not query user
        else:
            msg =f'Training will proceed until epoch {ask_epoch} then you will be asked to'
            print(msg )
            msg='enter H to halt training or enter an integer for how many more epochs to run then be asked again'
            print(msg)
            if self.dwell:
                msg='learning rate will be automatically adjusted during training'
                print(msg, (0,255,0))
        self.start_time= time.time() # set the time at which training started

    def on_train_end(self, logs=None):   # runs at the end of training
        msg=f'loading model with weights from epoch {self.best_epoch}'
        print(msg)
        model.set_weights(self.best_weights) # set the weights of the model to the best weights
        tr_duration=time.time() - self.start_time   # determine how long the training cycle lasted
        hours = tr_duration // 3600
        minutes = (tr_duration - (hours * 3600)) // 60
        seconds = tr_duration - ((hours * 3600) + (minutes * 60))
        msg = f'training elapsed time was {str(hours)} hours, {minutes:4.1f} minutes, {seconds:4.2f} seconds)'
        print (msg) # print out training duration time

    def on_epoch_end(self, epoch, logs=None):  # method runs on the end of each epoch
        vloss=logs.get('val_loss')  # get the validation loss for this epoch
        aloss=logs.get('loss')
        if epoch >0:
            deltav = self.lowest_vloss- vloss
            pimprov=(deltav/self.lowest_vloss) * 100
            self.plist.append(pimprov)
            deltaa=self.lowest_aloss-aloss
            aimprov=(deltaa/self.lowest_aloss) * 100
            self.alist.append(aimprov)
        else:
            pimprov=0.0
            aimprov=0.0
        if vloss< self.lowest_vloss:
            self.lowest_vloss=vloss
            self.best_weights=self.model.get_weights() # set best weights to model's initial weights
            self.best_epoch=epoch + 1
            msg=f'\n validation loss of {vloss:7.4f} is {pimprov:7.4f} % below lowest loss, saving weights from epoch {str(epoch + 1):3s} as best weights'
            print(msg) # green foreground
        else: # validation loss increased
            pimprov=abs(pimprov)
            msg=f'\n validation loss of {vloss:7.4f} is {pimprov:7.4f} % above lowest loss of {self.lowest_vloss:7.4f} keeping weights from epoch {str(self.best_epoch)} as best weights'
            print(msg) # yellow foreground
            if self.dwell: # if dwell is True when the validation loss increases the learning rate is automatically reduced and model weights are set to best weights
                lr=float(tf.keras.backend.get_value(self.model.optimizer.lr)) # get the current learning rate
                new_lr=lr * self.factor
                msg=f'learning rate was automatically adjusted from {lr:8.6f} to {new_lr:8.6f}, model weights set to best weights'
                print(msg) # cyan foreground
                tf.keras.backend.set_value(self.model.optimizer.lr, new_lr) # set the learning rate in the optimizer
                model.set_weights(self.best_weights) # set the weights of the model to the best weights

        if aloss< self.lowest_aloss:
            self.lowest_aloss=aloss
        if self.ask: # are the conditions right to query the user?
            if epoch + 1 ==self.ask_epoch: # is this epoch the one for quering the user?
                msg='\n Enter H to end training or  an integer for the number of additional epochs to run then ask again'
                print(msg) # cyan foreground
                ans=input()

                if ans == 'H' or ans =='h' or ans == '0': # quit training for these conditions
                    msg=f'you entered {ans},  Training halted on epoch {epoch+1} due to user input\n'
                    print(msg)
                    self.model.stop_training = True # halt training
                else: # user wants to continue training
                    self.ask_epoch += int(ans)
                    if self.ask_epoch > self.epochs:
                        print('\nYou specified maximum epochs of as ', self.epochs, ' cannot train for ', self.ask_epoch, flush =True)
                    else:
                        msg=f'you entered {ans} Training will continue to epoch {self.ask_epoch}'
                        print(msg) # cyan foreground
                        if self.dwell==False:
                            lr=float(tf.keras.backend.get_value(self.model.optimizer.lr)) # get the current learning rate
                            msg=f'current LR is  {lr:8.6f}  hit enter to keep  this LR or enter a new LR'
                            print(msg) # cyan foreground
                            ans=input(' ')
                            if ans =='':
                                msg=f'keeping current LR of {lr:7.5f}'
                                print(msg) # cyan foreground
                            else:
                                new_lr=float(ans)
                                tf.keras.backend.set_value(self.model.optimizer.lr, new_lr) # set the learning rate in the optimizer
                                msg=f' changing LR to {ans}'
                                print(msg) # cyan foreground

loading and data preprocessing

In [ ]:


# Load images from the 'items/' folder
temp = np.load('/content/drive/MyDrive/Copia di public_data.npz', allow_pickle = True)
img = temp["data"]
label = temp["labels"]
  # Normalize image pixel values to a float range [0, 1]
img = (img).astype(np.float32)

# set 0,1 label
for i in range(len(label)):
    if(label[i] == 'healthy'):
        label[i] = 0
    else:
        label[i] = 1

# cleaning images from trol and shrek
ref_img = img[58]
ref_img2 = img[2150]
c = 0
c2 = 0
rm_indexes = []
rm2_indexes = []
for i in range(0, len(img)):
    deviation = np.mean(np.abs(ref_img - img[i]))
    deviation2 = np.mean(np.abs(ref_img2 - img[i]))
    if(deviation == 0.0):
        #print(i)
        c += 1
        rm_indexes.append(i)
    elif(deviation2 == 0.0):
        c2 += 1
        rm2_indexes.append(i)

clean_img = np.delete(img, rm_indexes + rm2_indexes, axis=0)
clean_label = np.delete(label, rm_indexes + rm2_indexes, axis=0)


#DATA AUGMENTATION TO BALANCE CLASSES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

from keras.preprocessing.image import ImageDataGenerator
import numpy as np

# Create an instance of ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20,
#     shear_range=[0.3, 0.7],
#     zoom_range=[0.3, 0.9],
    horizontal_flip=True,
    vertical_flip=True,
    # brightness_range=[0, 0.7],
    fill_mode='reflect',
    preprocessing_function = orthogonal_rot
)

# Reshape the numpy array to fit the requirements of the flow method
x = clean_img[clean_label == 1]  # Replace 'your_numpy_array' with your actual numpy array

# Create a generator to augment the images
augmented_images = []
for x_batch in datagen.flow(x, batch_size=1, seed=42, shuffle=False):
    augmented_images.append(x_batch[0])
    if len(augmented_images) >= 1198:
        break

# Convert the list of augmented images to a numpy array
augmented_images_array = np.array(augmented_images)

clean_img = np.append(clean_img, augmented_images_array, axis = 0)
clean_label = np.append(clean_label, np.ones(1198))
print(clean_img.shape)
#!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
X = clean_img
y = clean_label

y = tfk.utils.to_categorical(y, 2) #one hot encodi

# Split data into train_val and test sets
#X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, random_state=seed, test_size=1000, stratify=np.argmax(y,axis=1))
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=seed, test_size=.15, stratify=np.argmax(y,axis=1))

# Further split train_val into train and validation sets
#X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, random_state=seed, test_size=1000, stratify=np.argmax(y_train_val,axis=1))

# Print shapes of the datasets
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
#print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")


######################OLD DATA AUGMENTATION -----way less efficient 

# BATCH_SIZE = 32
# BATCH_SIZE = 32

# from tensorflow.keras.preprocessing.image import ImageDataGenerator

# # # train generator with augmentation
# train_image_gen  = ImageDataGenerator(rotation_range=20, #maybe better not to rotate??
#                                       width_shift_range=0.1,
#                                       height_shift_range=0.1,
#                                       zoom_range=[0.5, 0.9],
#                                       shear_range=0.2,
#                                       brightness_range = [0.5, 1.5],
#                                       # vertical_flip=True,
#                                       # horizontal_flip=True,
#                                       fill_mode='nearest',
# #                                       validation_split = 0.15
#                                       preprocessing_function=add_noise
#                                       )


# # validation generator without augmentation
# # validation_image_gen = ImageDataGenerator(validation_split = 0.15,
# #                                           )

# train_dataset = train_image_gen.flow(x = X_train, y = y_train, seed = 42,
#                                      batch_size = BATCH_SIZE)
# validation_dataset = ImageDataGenerator().flow(X_val, y_val)

##########################################################################################




del img, clean_img, clean_label, temp, x


Data augmentations and train and validation tensors generators

In [ ]:
AUTO = tf.data.experimental.AUTOTUNE
HEIGHT = 96
WIDTH = 96

yy_train = np.argmax(y_train, axis = -1)
yy_val = np.argmax(y_val, axis = -1)
train_tf = tf.data.Dataset.from_tensor_slices((X_train, yy_train.astype(int)))#remove astype(int) at the end
validation_tf = tf.data.Dataset.from_tensor_slices((X_val, yy_val.astype(int)))

IMG_SHAPE = 96
BS = 128
AUTO = tf.data.experimental.AUTOTUNE

# Referred from: https://arxiv.org/pdf/2002.05709.pdf (Appendxi A
# corresponding GitHub: https://github.com/google-research/simclr/)
# I did not use all the augmentation policies proposed in the above paper

@tf.function
def custom_augment(image, label):
    # Random flips
    image = random_apply(tf.image.flip_left_right, image, p=0.4)
    image = random_apply(tf.image.flip_up_down, image, p = 0.2)
    image = random_apply(tf.image.rot90, image, p = 0.1)
#     image = random_apply(tf.image.random_crop()) #gotta resize
#     image = random_apply(crop_and_resize, image, p = 0.2)
    image = random_apply(cut_out, image, p = 0.8)
    image = random_apply(aug_shear, image, p = 0.4)
    image = random_apply(crop_img, image, p = 0.3)


    # Randomly apply transformation (color distortions) with probability p.
    image = random_apply(color_jitter, image, p=0.5)
#     image = random_apply(color_drop, image, p=0.2)
#     image = random_apply(cacca_su_img, image, p = 0.4)
    image = random_apply(tf.image.transpose, image, p = 0.5)
    return (image, label)

@tf.function
def color_jitter(x, s=0.3):
    # one can also shuffle the order of following augmentations
    # each time they are applied.
    x = tf.image.random_brightness(x, max_delta=0.8*s)
    x = tf.image.random_contrast(x, lower=1-0.8*s, upper=1+0.8*s)
    x = tf.image.random_saturation(x, lower=1-0.5*s, upper=1+0.5*s)
    x = tf.image.random_hue(x, max_delta=0.2*s)
    x = tf.clip_by_value(x, 0, 1)
    return x

@tf.function
def color_drop(x):
    x = tf.image.rgb_to_grayscale(x)
    x = tf.tile(x, [1, 1, 3])
    return x

@tf.function
def cut_out(x):
  return data_augment_cutout(x)

def data_augment_cutout(image, min_mask_size=(int(HEIGHT * .1), int(HEIGHT * .1)),
                        max_mask_size=(int(HEIGHT * .125), int(HEIGHT * .125))):
    p_cutout = tf.random.uniform([], 0, 1.0, dtype=tf.float32)

    if p_cutout > .85: # 10~15 cut outs
        n_cutout = tf.random.uniform([], 10, 15, dtype=tf.int32)
        image = random_cutout(image, HEIGHT, WIDTH,
                              min_mask_size=min_mask_size, max_mask_size=max_mask_size, k=n_cutout)
    elif p_cutout > .6: # 5~10 cut outs
        n_cutout = tf.random.uniform([], 5, 10, dtype=tf.int32)
        image = random_cutout(image, HEIGHT, WIDTH,
                              min_mask_size=min_mask_size, max_mask_size=max_mask_size, k=n_cutout)
    elif p_cutout > .25: # 2~5 cut outs
        n_cutout = tf.random.uniform([], 2, 5, dtype=tf.int32)
        image = random_cutout(image, HEIGHT, WIDTH,
                              min_mask_size=min_mask_size, max_mask_size=max_mask_size, k=n_cutout)
    else: # 1 cut out
        image = random_cutout(image, HEIGHT, WIDTH,
                              min_mask_size=min_mask_size, max_mask_size=max_mask_size, k=1)

    return image
def random_cutout(image, height, width, channels=3, min_mask_size=(10, 10), max_mask_size=(80, 80), k=1):
    assert height > min_mask_size[0]
    assert width > min_mask_size[1]
    assert height > max_mask_size[0]
    assert width > max_mask_size[1]
    for i in range(k):
      mask_height = tf.random.uniform(shape=[], minval=min_mask_size[0], maxval=max_mask_size[0], dtype=tf.int32)
      mask_width = tf.random.uniform(shape=[], minval=min_mask_size[1], maxval=max_mask_size[1], dtype=tf.int32)

      pad_h = height - mask_height
      pad_top = tf.random.uniform(shape=[], minval=0, maxval=pad_h, dtype=tf.int32)
      pad_bottom = pad_h - pad_top

      pad_w = width - mask_width
      pad_left = tf.random.uniform(shape=[], minval=0, maxval=pad_w, dtype=tf.int32)
      pad_right = pad_w - pad_left

      cutout_area = tf.zeros(shape=[mask_height, mask_width, channels], dtype=tf.uint8)

      cutout_mask = tf.pad([cutout_area], [[0,0],[pad_top, pad_bottom], [pad_left, pad_right], [0,0]], constant_values=1)
      cutout_mask = tf.squeeze(cutout_mask, axis=0)
      image = tf.multiply(tf.cast(image, tf.float32), tf.cast(cutout_mask, tf.float32))

    return image

@tf.function
def aug_shear(x):
  p_shear = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
  if p_shear > .5:
      x = transform_shear(x, HEIGHT, shear=20.)
  else:
      x = transform_shear(x, HEIGHT, shear=-20.)
  return x

def transform_shear(image, height, shear):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly sheared
    DIM = height
    XDIM = DIM%2 #fix for size 331

    shear = shear * tf.random.uniform([1],dtype='float32')
    shear = math.pi * shear / 180.

    # SHEAR MATRIX
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape(tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3])

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )

    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = K.dot(shear_matrix,tf.cast(idx,dtype='float32'))
    idx2 = K.cast(idx2,dtype='int32')
    idx2 = K.clip(idx2,-DIM//2+XDIM+1,DIM//2)

    # FIND ORIGIN PIXEL VALUES
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image, tf.transpose(idx3))

    return tf.reshape(d,[DIM,DIM,3])

@tf.function
def crop_img(x):
  p_crop = tf.random.uniform([], 0, 1.0, dtype=tf.float32)
  if p_crop > .6:
    if p_crop > .85:
        x = tf.image.central_crop(x, central_fraction=.7)
    else:
        x = tf.image.central_crop(x, central_fraction=.8)
  else:
    crop_size = tf.random.uniform([], int(HEIGHT*.6), HEIGHT, dtype=tf.int32)
    x = tf.image.random_crop(x, size=[crop_size, crop_size, 3])
  x = tf.image.resize(x, size=[HEIGHT, WIDTH])
  return x



@tf.function
def random_apply(func, x, p):
    return tf.cond(
        tf.less(tf.random.uniform([], minval=0, maxval=1, dtype=tf.float32),
                tf.cast(p, tf.float32)),
        lambda: func(x),
        lambda: x)

# # @tf.function
# def cacca_su_img(imageee):
#     imagee = 255*imageee.numpy()
#     lista = [20, 25, 30, 35]
#     l = random.choice(lista)
#     box = np.zeros(l**2).reshape(l, l)
#     xx, yy = np.random.randint(0,95-l, size=2)
#     for i in range(3):
#         imagee[xx:xx+l, yy:yy+l, i] = box

#     imagee = tf.convert_to_tensor(imagee)

#     return imagee/255



@tf.function
def preprocess_image(image, label):
  # image = image/255
  image = tf.image.convert_image_dtype(image, tf.float32)
  #     image = tf.image.resize(image, (IMG_SHAPE, IMG_SHAPE))
  image = image/255
  return (image, label)





@tf.function
def preprocess_image_class(image, label):
  image = image/255
  label = tf.one_hot()
  return (image, label)

train_tf = (
    train_tf
    .map(preprocess_image, num_parallel_calls=AUTO)
    .shuffle(100)
    .map(custom_augment, num_parallel_calls=AUTO)
    .batch(BS)
    .prefetch(AUTO)
)

validation_tf = (
    validation_tf
    .map(preprocess_image, num_parallel_calls=AUTO)
    .shuffle(100)
    .batch(BS)#BS
    .prefetch(AUTO)
)
#IF YOU WANT TO DO CLASSIFICATION AND NOT SUPERVISED CONTRASTIVE TRAINING, RUN THIS!
# train_tf = (train_tf.map(lambda x, y: (x, tf.one_hot(y, depth =2)))) #let's go back to categorical values
# validation_tf = (validation_tf.map(lambda x, y: (x, tf.one_hot(y, depth = 2))))


In [ ]:
#example of images augmented
image_batch, label_batch = next(iter(train_tf))

plt.figure(figsize=(10, 10))
for n in range(25):
    ax = plt.subplot(5, 5, n+1)
    plt.imshow(image_batch[n])
    plt.title(label_batch[n].numpy())
    plt.axis('off')

# Supervised Contrastive Learning 

## Training the encoder: EffnetV2L

In [ ]:
IMG_SHAPE = 96
class UnitNormLayer(tf.keras.layers.Layer):
    '''Normalize vectors (euclidean norm) in batch to unit hypersphere.
    '''
    def __init__(self):
        super(UnitNormLayer, self).__init__()

    def call(self, input_tensor):
        norm = tf.norm(input_tensor, axis=1)
        return input_tensor / tf.reshape(norm, [-1, 1])

    # Encoder Network
def encoder_net():
  inputs = Input((IMG_SHAPE, IMG_SHAPE, 3))
  normalization_layer = UnitNormLayer()

  encoder = tf.keras.applications.EfficientNetV2L(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(96, 96, 3),
    pooling='max',
    classes=1000,
    classifier_activation="softmax",
    include_preprocessing=True,
    )
 
  embeddings = encoder(inputs, training=True)
  # embeddings = GlobalAveragePooling2D()(embeddings)
  norm_embeddings = normalization_layer(embeddings)

  encoder_network = Model(inputs, norm_embeddings)

  return encoder_network

# Projector Network
def projector_net():
	projector = tf.keras.models.Sequential([
		Dense(128, activation="relu"),
		UnitNormLayer()
	])

	return projector


In [ ]:
decay_steps = 1000 #COSINE DECAYING LEARNING RATE SCHEDULING 
lr_decayed_fn = tf.keras.experimental.CosineDecay(
    initial_learning_rate=0.001, decay_steps=decay_steps)
optimizer = tf.keras.optimizers.Adam(lr_decayed_fn)

In [ ]:
encoder_r = encoder_net()
projector_z = projector_net()

@tf.function
def train_step(images, labels): #just for the single batch
	with tf.GradientTape() as tape: #starts a context for automatic differentiation
        #it defines a block of operations to be recorded for AD.
        #any operations performed on tensors  are recorded on the tape
        #so the gradients can be computed wrto the recorded computations
		r = encoder_r(images, training=True)
		z = projector_z(r, training=True)
		loss = losses.max_margin_contrastive_loss(z, labels, metric='cosine')
#         loss = losses.SupervisedContrastiveLoss(temperature)

	gradients = tape.gradient(loss,
		encoder_r.trainable_variables + projector_z.trainable_variables)
	optimizer.apply_gradients(zip(gradients,
		encoder_r.trainable_variables + projector_z.trainable_variables))

	return loss

In [ ]:
EPOCHS = 20
LOG_EVERY = 2
train_loss_results = []

start = time.time()
for epoch in tqdm(range(EPOCHS)):
	epoch_loss_avg = tf.keras.metrics.Mean()

	for (images, labels) in train_tf:
		loss = train_step(images, labels)
		epoch_loss_avg.update_state(loss)#updates for every epoch (training set)

	train_loss_results.append(epoch_loss_avg.result())

	if epoch % LOG_EVERY == 0:
		print("Epoch: {} Loss: {:.3f}".format(epoch, epoch_loss_avg.result()))

end = time.time()

with plt.xkcd():
    plt.plot(train_loss_results)
    plt.title("Supervised Contrastive Loss")
    plt.show()

### PLOT OF THE EMBEDDINGS 

In [ ]:
oof_reg_pred = []; embeddings = []
true_labels = np.argmax(y, axis = -1)
embeddings = projector_z.predict(encoder_r.predict(X))


In [ ]:
import pandas as pd
from sklearn.manifold import TSNE
def visualize_embeddings(embeddings, labels, figsize=(8, 8)):
    # Extract TSNE values from embeddings
    embed2D = TSNE(n_components=2, n_jobs=-1, random_state=seed).fit_transform(embeddings)
    embed2D_x = embed2D[:,0]
    embed2D_y = embed2D[:,1]

    # Create dataframe with labels and TSNE values
    df_embed = pd.DataFrame({'labels': labels})
    df_embed = df_embed.assign(x=embed2D_x, y=embed2D_y)
    # df_embed.to_csv('/content/drive/MyDrive/embed_effnetv2L.csv', index=True)
    # Create classes dataframes
    df_embed_healthy = df_embed[df_embed['labels'] == 0]
    df_embed_unhealthy = df_embed[df_embed['labels'] == 1]


    # Plot embeddings
    plt.figure(figsize=figsize)
    plt.scatter(df_embed_healthy['x'], df_embed_healthy['y'],color='red',s=10,label='healthy')
    plt.scatter(df_embed_unhealthy['x'], df_embed_unhealthy['y'],color='blue',s=10,label='unhealthy')

    plt.legend()
    plt.show()

visualize_embeddings(embeddings, list(true_labels))

## CLASSIFICATION

In [ ]:
train_tf = (train_tf.map(lambda x, y: (x, tf.one_hot(y, depth =2)))) #let's go back to categorical values, CRUCIAL TO DO CLASSIFICATION 
validation_tf = (validation_tf.map(lambda x, y: (x, tf.one_hot(y, depth = 2))))

In [ ]:
IMG_SHAPE = 96
def supervised_model():
    inputs = Input((IMG_SHAPE, IMG_SHAPE, 3))
    encoder_r.trainable = False

    r = encoder_r(inputs, training=False)
    # x = layers.GlobalAveragePooling2D(name = "gap")(x)
    x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name = "batch")(r)


    x = layers.Dense(256,activation='relu')(x)
    x = layers.Dense(256, kernel_initializer=tf.keras.initializers.HeUniform(seed=42))(x)
    x = layers.BatchNormalization(name = "batch2")(x)
    x = layers.Dropout(0.2, seed = 42, name = "dropout")(x)

    x = layers.Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                        bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
    x = layers.BatchNormalization(name = "batch3")(x)
    x = layers.Dropout(0.3, seed = 42, name = "dropout2")(x)

    x = layers.Dense(128, activation = "relu", name = "last_dense")(x)
    x = layers.BatchNormalization(name = "batch4")(x)
    x = layers.Dropout(0.3, seed = 42, name = "dropout3")(x)

    outputs = Dense(2, activation = "softmax")(x)



    supervised_model = Model(inputs, outputs)

    return supervised_model

In [ ]:
from tensorflow.keras import layers
model = supervised_model()

model.compile(optimizer=tf.keras.optimizers.Adam(),
  # loss = 'binary_crossentropy',
	loss=tfk.losses.CategoricalCrossentropy(),
	metrics=["accuracy"])

es = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10,
	restore_best_weights=True, verbose=1)

model.summary()


epochs=100
ask_epoch=30
ask=LR_ASK(model, epochs,  ask_epoch, factor = 0.8) #FACTOR MEASURES BY HOW MUCH WE DECREASE THE LEARNING RATE
callbacks=[ask]

history=model.fit(x=train_tf,  epochs=epochs, verbose=1, callbacks=callbacks,  validation_data=validation_tf,
               validation_steps=None,  shuffle=False,  initial_epoch=0)

# TRANSFER LEARNING AND FINE TUINING

In [ ]:
#RUN IF NOT RUN BEFORE: puts predictions back in one hot encoding format
# train_tf = (train_tf.map(lambda x, y: (x, tf.one_hot(y, depth =2)))) #let's go back to categorical values
# validation_tf = (validation_tf.map(lambda x, y: (x, tf.one_hot(y, depth = 2))))

In [ ]:

convlarge = tf.keras.applications.ConvNeXtBase( #here just change with other models from keras.applications
    include_top=False,
    include_preprocessing=True,
    weights="imagenet",
    input_tensor=None,
    input_shape= (96,96, 3),
    pooling="max",
    classes=2,
    classifier_activation="softmax",
)

convlarge.trainable = False


from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
# Build the neural network layer by layer
inputs = tfkl.Input(shape=(96, 96, 3))

# x = img_augmentation(inputs)
x = convlarge(inputs*255) #iamges in train_tf 
# x = layers.GlobalAveragePooling2D(name = "gap")(x)
x=BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, name = "batch")(x)


x = layers.Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                    bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
# x = layers.Dense(256, kernel_initializer=tf.keras.initializers.HeUniform(seed=42))(x)
x = layers.BatchNormalization(name = "batch2")(x)
x = layers.Dropout(0.3, seed = 42, name = "dropout")(x)

x = layers.Dense(128, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                    bias_regularizer=regularizers.l1(0.006) ,activation='relu')(x)
x = layers.BatchNormalization(name = "batch3")(x)
x = layers.Dropout(0.3, seed = 42, name = "dropout2")(x)

outputs = tfkl.Dense(units=2, activation='softmax',name='Output2')(x)  #you cannnot use softmax with only one neuron since it normalizes over the output neurons
model = tfk.Model(inputs, outputs)
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adamax(learning_rate=0.01), metrics=['accuracy'])
model.summary()

In [ ]:
epochs=100
ask_epoch=20 
ask=LR_ASK(model, epochs,  ask_epoch)
callbacks=[ask]

history=model.fit(x=train_dataset,  epochs=epochs, verbose=1, callbacks=callbacks,  validation_data=validation_dataset,
               validation_steps=None,  shuffle=False,  initial_epoch=0)

## Fine Tuning procedure
$32layers \rightarrow 64layers \rightarrow 96layers \rightarrow allLayers$

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

unfreeze = 32 #change it as you go on with the procedure. Go lower if 32 is already too much

model.get_layer('convnext_base').trainable = True
for i, layer in enumerate(model.get_layer('convnext_base').layers[:-unfreeze]):
    layer.trainable = False

# make sure BatchNorm layers are frozen
for i, layer in enumerate(model.get_layer('convnext_base').layers):
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False

model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adamax(0.0001), metrics=['accuracy'])


model.summary()

In [ ]:
epochs=100
ask_epoch=10
ask=LR_ASK(model, epochs,  ask_epoch, factor = 0.55)
callbacks=[ask]
history=model.fit(x=train_dataset,  epochs=epochs, verbose=1, callbacks=callbacks,  validation_data=validation_dataset,
                validation_steps=None,  shuffle=False,  initial_epoch=0)

# Ensemble

In [ ]:
#RUN IF NOT RUN BEFORE: puts predictions back in one hot encoding format
# train_tf = (train_tf.map(lambda x, y: (x, tf.one_hot(y, depth =2)))) #let's go back to categorical values
# validation_tf = (validation_tf.map(lambda x, y: (x, tf.one_hot(y, depth = 2))))

In [ ]:
model_conv = tfk.models.load_model('/content/drive/MyDrive/Homework’s/models/tensor_conv_base_full3')
model_SCL = tfk.models.load_model('/content/drive/MyDrive/Homework’s/models/essnetv2L_more_augmentation')
model_small = tfk.models.load_model('/content/drive/MyDrive/Homework’s/models/conv_small_64')

model_conv._name = 'model_conv'
model_SCL._name = 'model_SCL'
model_small._name = 'model_small'

In [ ]:
from tensorflow.keras.layers import Layer, Concatenate
def weight_init(shape =(1,1,2), weights=[1, 1, 1.5], dtype=tf.float32): #convnest_small has more weight (found to give better results)
    return tf.constant(np.array(weights).reshape(shape), dtype=dtype)

class WeightedAverage(Layer):

    def __init__(self):
        super(WeightedAverage, self).__init__()

    def build(self, input_shape):

        self.W = self.add_weight(
                    shape=(1,1,len(input_shape)),
                    initializer=weight_init,
                    dtype=tf.float32,
                    trainable=True, name = 'cacca')
    def call(self, inputs):

        inputs = [tf.expand_dims(i, -1) for i in inputs]
        inputs = Concatenate(axis=-1)(inputs)
        weights = tf.nn.softmax(self.W, axis=-1)

        return tf.reduce_mean(weights*inputs, axis=-1)

In [ ]:
from tensorflow.keras.layers import Layer, Concatenate
models = [model_conv, model_SCL, model_small]

inputs = tfk.Input(shape=(96, 96, 3))
outputs = [model(inputs) for model in models]
outputs_ensemble = WeightedAverage()(outputs)
model_ensemble = tfk.Model(inputs=inputs, outputs=outputs_ensemble, name='ensemble_base_large')

model_ensemble.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adamax(learning_rate=0.01), metrics=['accuracy'])


# CNN with CBAM Architecture

In [ ]:
import keras
from keras.layers import GlobalAveragePooling2D, GlobalMaxPooling2D, Reshape, Dense, multiply, Permute, Concatenate, Conv2D, Add, Activation, Lambda
from keras import backend as K
from keras.activations import sigmoid
from keras import layers
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras import initializers
from keras.models import Sequential
def cbam_block(cbam_feature, ratio=8):
    """Contains the implementation of Convolutional Block Attention Module(CBAM) block.
    As described in https://arxiv.org/abs/1807.06521.
    """

    cbam_feature = channel_attention(cbam_feature, ratio)
    cbam_feature = spatial_attention(cbam_feature)
    return cbam_feature

def channel_attention(input_feature, ratio=8):

    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    channel = input_feature.shape[channel_axis]

    shared_layer_one = Dense(channel//ratio,
                             activation='relu',
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')
    shared_layer_two = Dense(channel,
                             kernel_initializer='he_normal',
                             use_bias=True,
                             bias_initializer='zeros')

    avg_pool = GlobalAveragePooling2D()(input_feature)
    avg_pool = Reshape((1,1,channel))(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel)
    avg_pool = shared_layer_one(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel//ratio)
    avg_pool = shared_layer_two(avg_pool)
    assert avg_pool.shape[1:] == (1,1,channel)

    max_pool = GlobalMaxPooling2D()(input_feature)
    max_pool = Reshape((1,1,channel))(max_pool)
    assert max_pool.shape[1:] == (1,1,channel)
    max_pool = shared_layer_one(max_pool)
    assert max_pool.shape[1:] == (1,1,channel//ratio)
    max_pool = shared_layer_two(max_pool)
    assert max_pool.shape[1:] == (1,1,channel)

    cbam_feature = Add()([avg_pool,max_pool])
    cbam_feature = Activation('sigmoid')(cbam_feature)

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)

    return multiply([input_feature, cbam_feature])

def spatial_attention(input_feature):
    kernel_size = 7

    if K.image_data_format() == "channels_first":
        channel = input_feature.shape[1]
        cbam_feature = Permute((2,3,1))(input_feature)
    else:
        channel = input_feature.shape[-1]
        cbam_feature = input_feature

    avg_pool = Lambda(lambda x: K.mean(x, axis=3, keepdims=True))(cbam_feature)
    assert avg_pool.shape[-1] == 1
    max_pool = Lambda(lambda x: K.max(x, axis=3, keepdims=True))(cbam_feature)
    assert max_pool.shape[-1] == 1
    concat = Concatenate(axis=3)([avg_pool, max_pool])
    assert concat.shape[-1] == 2
    cbam_feature = Conv2D(filters = 1,
                    kernel_size=kernel_size,
                    strides=1,
                    padding='same',
                    activation='sigmoid',
                    kernel_initializer='he_normal',
                    use_bias=False)(concat)
    assert cbam_feature.shape[-1] == 1

    if K.image_data_format() == "channels_first":
        cbam_feature = Permute((3, 1, 2))(cbam_feature)

    return multiply([input_feature, cbam_feature])

In [ ]:
train_batches = train_dataset
validation_batches = validation_dataset
IMG_SIZE = 96
n_classes = 2
EPOCHS = 50
BATCH_SIZE = 8
IMG_SIZE = 224
n_classes = 4
METRICS = [tfk.metrics.CategoricalAccuracy(name='accuracy'),
           tfk.metrics.Precision(name='precision'),
           tfk.metrics.Recall(name='recall'),
           tfk.metrics.AUC(name='auc')]

inp = layers.Input(shape=(IMG_SIZE,IMG_SIZE,3))
x1 = layers.Resizing(IMG_SIZE, IMG_SIZE)(inp)
# x2 = layers.Rescaling(1./255)(x1)
x3 = layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu',
                   padding = 'same', input_shape=(IMG_SIZE,IMG_SIZE,3))(x1)
x4 = cbam_block(x3)
x5 = layers.BatchNormalization(axis=-1)(x4)
x6 = layers.MaxPool2D(pool_size=(2, 2))(x5)
x7 = layers.Conv2D(filters=64, kernel_size=(3, 3),
                   activation='relu', padding = 'same')(x6)
x8 = cbam_block(x7)
x9 = layers.BatchNormalization(axis=-1)(x8)
x10 = layers.MaxPool2D(pool_size=(2, 2))(x9)
x11 = layers.Conv2D(filters=128, kernel_size=(3, 3),
                    activation='relu', padding = 'same')(x10)
x12 = cbam_block(x11)
x13 = layers.BatchNormalization(axis=-1)(x12)
x14 = layers.MaxPool2D(pool_size=(2, 2))(x13)
x15 = layers.Conv2D(filters=256, kernel_size=(3, 3),
                      activation='relu', padding = 'same')(x14)
x16 = cbam_block(x15)
x17 = layers.BatchNormalization(axis=-1)(x16)
x18 = layers.MaxPool2D(pool_size=(2, 2))(x17)
y1 = layers.Flatten()(x18)
y2 = layers.Dense(300, activation='relu')(y1)
y3 = layers.Dense(150, activation='relu')(y2)
y4 = layers.Dropout(0.25)(y3)
y = layers.Dense(n_classes, activation='softmax')(y4)


model = tf.keras.Model(inp, y)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                   loss='categorical_crossentropy',
                   metrics=METRICS)
model.summary()

In [ ]:
epochs=100
ask_epoch=10
ask=LR_ASK(model, epochs,  ask_epoch, factor = 0.65)
callbacks=[ask]
history=model.fit(x=train_dataset,  epochs=epochs, verbose=1, callbacks=callbacks,  validation_data=validation_dataset,
               validation_steps=None,  shuffle=False,  initial_epoch=0)